In [1]:
import graphlab as gl
import graphlab.aggregate
import datetime
import time

In [2]:
data = gl.SFrame("https://s3.amazonaws.com/dato-datasets/churn-prediction/online_retail.csv")
data

This non-commercial license of GraphLab Create is assigned to voelunteer@gmail.com and will expire on December 23, 2016. For commercial licensing options, visit https://dato.com/buy/.


2016-04-08 22:25:29,459 [INFO] graphlab.cython.cy_server, 176: GraphLab Create v1.8.5 started. Logging: /tmp/graphlab_server_1460168728.log


Downloading https://s3.amazonaws.com/dato-datasets/churn-prediction/online_retail.csv to /var/tmp/graphlab-voelunteer/2063/27684c90-2e41-4c09-bc86-006c2943696f.csv

Finished parsing file https://s3.amazonaws.com/dato-datasets/churn-prediction/online_retail.csv

Parsing completed. Parsed 100 lines in 1.67793 secs.

------------------------------------------------------
Inferred types from first line of file as 
column_type_hints=[int,str,str,int,str,float,int,str]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file https://s3.amazonaws.com/dato-datasets/churn-prediction/online_retail.csv

Parsing completed. Parsed 541909 lines in 1.62862 secs.

InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
536365,85123A,WHITE HANGING HEARTT-LIGHT HOLDER ...,6,12/1/10 8:26,2.55,17850,United Kingdom
536365,71053,WHITE METAL LANTERN,6,12/1/10 8:26,3.39,17850,United Kingdom
536365,84406B,CREAM CUPID HEARTS COATHANGER ...,8,12/1/10 8:26,2.75,17850,United Kingdom
536365,84029G,KNITTED UNION FLAG HOTWATER BOTTLE ...,6,12/1/10 8:26,3.39,17850,United Kingdom
536365,84029E,RED WOOLLY HOTTIE WHITEHEART. ...,6,12/1/10 8:26,3.39,17850,United Kingdom
536365,22752,SET 7 BABUSHKA NESTINGBOXES ...,2,12/1/10 8:26,7.65,17850,United Kingdom
536365,21730,GLASS STAR FROSTEDT-LIGHT HOLDER ...,6,12/1/10 8:26,4.25,17850,United Kingdom
536366,22633,HAND WARMER UNION JACK,6,12/1/10 8:28,1.85,17850,United Kingdom
536366,22632,HAND WARMER RED POLKA DOT,6,12/1/10 8:28,1.85,17850,United Kingdom
536367,84879,ASSORTED COLOUR BIRDORNAMENT ...,32,12/1/10 8:34,1.69,13047,United Kingdom


In [3]:
data = data.remove_columns(['InvoiceNo', 'Description'])
data

StockCode,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
85123A,6,12/1/10 8:26,2.55,17850,United Kingdom
71053,6,12/1/10 8:26,3.39,17850,United Kingdom
84406B,8,12/1/10 8:26,2.75,17850,United Kingdom
84029G,6,12/1/10 8:26,3.39,17850,United Kingdom
84029E,6,12/1/10 8:26,3.39,17850,United Kingdom
22752,2,12/1/10 8:26,7.65,17850,United Kingdom
21730,6,12/1/10 8:26,4.25,17850,United Kingdom
22633,6,12/1/10 8:28,1.85,17850,United Kingdom
22632,6,12/1/10 8:28,1.85,17850,United Kingdom
84879,32,12/1/10 8:34,1.69,13047,United Kingdom


In [4]:
import dateutil
from dateutil import parser
def _unix_timestamp_to_datetime(x):
    import datetime
    import pytz
    return dateutil.parser.parse(x)

data['InvoiceDate'] = data['InvoiceDate'].apply(_unix_timestamp_to_datetime)

In [5]:
(train, valid) = gl.churn_predictor.random_split(data, user_id = 'CustomerID', fraction = 0.9, seed = 12)
train_trial = gl.TimeSeries(train, index = 'InvoiceDate')
valid_trial = gl.TimeSeries(valid, index = 'InvoiceDate')

In [6]:
userdata = gl.SFrame("https://s3.amazonaws.com/dato-datasets/churn-prediction/online_retail_side_data_extended.csv")
userdata

Downloading https://s3.amazonaws.com/dato-datasets/churn-prediction/online_retail_side_data_extended.csv to /var/tmp/graphlab-voelunteer/2063/8f6367e9-d5e4-4516-adc3-b6d6d9c5ca27.csv

Finished parsing file https://s3.amazonaws.com/dato-datasets/churn-prediction/online_retail_side_data_extended.csv

Parsing completed. Parsed 100 lines in 0.044857 secs.

------------------------------------------------------
Inferred types from first line of file as 
column_type_hints=[int,str,int,str]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file https://s3.amazonaws.com/dato-datasets/churn-prediction/online_retail_side_data_extended.csv

Parsing completed. Parsed 4380 lines in 0.017916 secs.

CustomerID,Gender,Age,Country
13097,Male,57,United Kingdom
16817,Male,57,United Kingdom
14499,Male,61,United Kingdom
16185,Male,33,United Kingdom
14285,Male,33,United Kingdom
16837,Male,57,United Kingdom
13969,Male,41,United Kingdom
12831,Male,45,United Kingdom
16697,Male,57,United Kingdom
17671,Male,45,United Kingdom
